In [ ]:
# The first thing we need to do is import the required libraries

import csv
import math
import random

In [ ]:
class Decisiond_tree():
    d_tree = {}

    def learn(self, training_data, features, target):
        # def train(training_data, features, target):
      training_data = [item for item in training_data]
      vals = [record[features.index(target)] for record in training_data]
      frequency = {key:0 for key in [item[-1] for item in training_data]}
      idx = features.index(target)
      for tup in training_data:
          frequency[tup[idx]] += 1
      max, major = 0, 0
      for key in frequency.keys():
          if frequency[key]>max:
              max = frequency[key]
              major = key

      if not training_data:
          return major
      elif (len(features) - 1) <= 0:
          return major
      elif vals.count(vals[0]) == len(vals):
          return vals[0]
      else:
          best = features[0]
          minimum_gini = 10000;

          for attr in features:
              new_gini_index = get_gini(features, training_data, attr, target)
              if new_gini_index<minimum_gini:
                  minimum_gini = new_gini_index
                  best = attr
          d_tree = {best:{}}

          idx = features.index(best)
          got_vals = []
          for entry in training_data:
              if entry[idx] not in got_vals:
                  got_vals.append(float(entry[idx]))
          for val in got_vals: #get_values(data, features, best):
              new_data = [[entry[i] for i in range(0, len(entry)) if i != idx] for entry in training_data if entry[idx] == val]
              newAttr = features[:]
              newAttr.remove(best)
              subtree = self.learn(new_data, newAttr, target)
              d_tree[best][val] = subtree
      return d_tree
      
    def classify(self, test_instance, features):
        # Followinf node is to make sure that we create an entity as node
        # the node will store the name of a node in the tree and its children
        class Node():
          def __init__(self, val, dictionary):
              self.value = val
              if (isinstance(dictionary, dict)):
                  self.children = dictionary.keys()
        result = 0 # baseline: always classifies as 0
        # create a copy of d_tree in the varable temp_dict
        temp_dict = self.d_tree.copy()

        # Traverse the tree untill reaching the leaf node
        while(isinstance(temp_dict, dict)):
          # get to the root node of the currect subtree
          root = Node(list(temp_dict.keys())[0], temp_dict[list(temp_dict.keys())[0]])
          # Store the children of the current node in the temp_dict variable
          temp_dict = temp_dict[list(temp_dict.keys())[0]]
          # store the idx of the attribute of the current node and store it in idx variable
          idx = features.index(root.value)
          # Pick the value from the test tuple for the variable location stored in idx variable
          value = test_instance[idx]
          # store the keys of the 
          if value in temp_dict.keys():
            child = Node(value, temp_dict[value])
            result = temp_dict[value]
            temp_dict = temp_dict[value]
          else:
            result = None
            break
        return result

    def gini_target(self, features, data, target_attributes):
        # Set up a dictionary to store the no of occurences of the target attribute
        freq = {}
        # Set the sum of probabilitys
        sum_p2 = 0.0
        i = features.index(target_attributes) - 1
        for entry in data:
            if (entry[i] in freq.keys()):
                freq[entry[i]] += 1.0
            else:
                freq[entry[i]]  = 1.0
        for freq in freq.values():
            sum_p2 += (freq/len(data))**2
        gini = 1 - sum_p2
        return gini

    def get_gini(self, features, data, attr, target_attributes):
        # This function will help to calculate the Gini index of the part of data
        # Using this function we will calculate the Gini index for each split made by each attribute
        var_occur_n = {}

        # Subset entropy variable will contain the sum of entropy of each and every split of the attribute
        gini_sum = 0.0
        # store the idx of the current attribute column
        i = features.index(attr)

        # Iterating over the the data provided we will canculate the frequency of occurence of 
        # the unique elements of the attribute
        for entry in data:
            # for each entry in the data update their frequency
            # If the entry is listed as a key then increment the frequency
            # On the other hand if the entry is not listed then add it as a new element
            if (entry[i] in var_occur_n.keys()):
                var_occur_n[entry[i]] += 1.0
            else:
                var_occur_n[entry[i]]  = 1.0

        # For each key in the dictionary canculate the entropy of corresponding subset of data
        # then add those entropy values in the subset_entropy
        for val in var_occur_n.keys():
            Probability_val        = var_occur_n[val] / sum(var_occur_n.values())
            dataSubset     = [entry for entry in data if entry[i] == val]
            gini_sum += Probability_val * gini_target(features, dataSubset, target_attributes)

        return gini_sum
